In [3]:
from langgraph.graph import StateGraph, START,END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import os
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
#Define state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
    step3: str

In [6]:
def step_1(state: CrashState):
    print("Step 1 executed")
    
    return {"step1": "done", "input": state['input']}

In [7]:
def step_2(state: CrashState):
    print("Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(30) # simulate long-running hang
    return {"step2": "done"}


In [8]:
def step_3(state: CrashState):
    print("Step 3 executed")
    return {"done":True}

In [9]:
# build the graph

graph = StateGraph(CrashState)

#add nodes
graph.add_node("step_1",step_1)
graph.add_node("step_2",step_2)
graph.add_node("step_3",step_3)

# add edges
graph.add_edge(START, "step_1")
graph.add_edge("step_1","step_2")
graph.add_edge("step_2","step_3")
graph.add_edge("step_3",END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer = checkpointer)





In [15]:
try:
    print(" Running graph: please manually interrupt during step 2..")
    workflow.invoke({"input": "start"}, config = {"configurable" : {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("Kernel manually interrupted (crash simulated).")
    

 Running graph: please manually interrupt during step 2..
Step 1 executed
Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
Kernel manually interrupted (crash simulated).


In [16]:
workflow.get_state({"configurable": {"thread_id": 'thread-1'}})

StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0572-299c-60e8-8004-5c8cddc1b9d0'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2025-12-03T14:48:30.971883+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0572-2996-61c2-8003-5896fce5e5e9'}}, tasks=(PregelTask(id='549699f4-e42f-1fd6-dcec-877276da4826', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [18]:
list(workflow.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0572-299c-60e8-8004-5c8cddc1b9d0'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2025-12-03T14:48:30.971883+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0572-2996-61c2-8003-5896fce5e5e9'}}, tasks=(PregelTask(id='549699f4-e42f-1fd6-dcec-877276da4826', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_1',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0572-2996-61c2-8003-5896fce5e5e9'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-12-03T14:48:30.969460+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkp

In [19]:
final_state = workflow.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n Final state:", final_state)

Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
Step 3 executed

 Final state: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [24]:
workflow.get_state({"configurable": {"thread_id": 'thread-1'}})

StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0576-6337-6ffc-8006-bb7c4f2f4af2'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-12-03T14:50:24.386831+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0576-6332-699a-8005-499d8361d03e'}}, tasks=(), interrupts=())

In [28]:
list(workflow.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0576-6337-6ffc-8006-bb7c4f2f4af2'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-12-03T14:50:24.386831+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0576-6332-699a-8005-499d8361d03e'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0576-6332-699a-8005-499d8361d03e'}}, metadata={'source': 'loop', 'step': 5, 'parents': {}}, created_at='2025-12-03T14:50:24.384594+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d0572-299c-60e8-8004-5c8cddc1b9d0'}}, tasks=(PregelTask(id='e8ac8592-35f6-db60-991f-79762097998